# ReAct Agent

In this notebook we will look into creating ReAct Agent over tools.

1. ReAct Agent over simple calculator tools.
2. ReAct Agent over QueryEngine (RAG) tools.

### Installation

In [ ]:
!pip install llama-index
!pip install llama-index-llms-anthropic
!pip install llama-index-embeddings-huggingface

### Setup API Keys

In [2]:
# llama-parse is async-first, running the async code in a notebook requires the use of nest_asyncio
import nest_asyncio
nest_asyncio.apply()

import os

# Using Anthropic LLM API for LLM
os.environ['ANTHROPIC_API_KEY'] = 'YOUR ANTHROPIC API KEY'

### Set LLM and Embedding model

In [3]:
from llama_index.llms.anthropic import Anthropic
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [4]:
llm = Anthropic(temperature=0.0, model='claude-2.1')
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

In [5]:
from llama_index.core import Settings
Settings.llm = llm
Settings.embed_model = embed_model

## ReAct Agent over Tools

### Define Tools

In [6]:
from llama_index.core.agent import ReActAgent
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool

In [7]:
def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b

def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)
multiply_tool = FunctionTool.from_defaults(fn=multiply)

### Create ReAct Agent 

Create agent over tools and test out queries

In [ ]:
agent = ReActAgent.from_tools([multiply_tool, add_tool], llm=llm, verbose=True)

In [8]:
response = agent.chat("What is 20+(2*4)? Calculate step by step ")

Thought: I need to break this down step-by-step to solve it.
Action: multiply
Action Input: {'a': 2, 'b': 4}
Observation: 8
Thought: I multiplied 2 and 4 to get 8.
Action: add
Action Input: {'a': 20, 'b': 8}
Observation: 28
Thought: I added 20 and 8 to get the final answer of 28.
Answer: 20+(2*4) = 20+8 = 28


In [9]:
print(response)

20+(2*4) = 20+8 = 28


### Visit Prompts

You can check prompts that the agent used to select the tools.

In [10]:
prompt_dict = agent.get_prompts()
for k, v in prompt_dict.items():
    print(f"Prompt: {k}\n\nValue: {v.template}")

Prompt: agent_worker:system_prompt

Value: 
You are designed to help with a variety of tasks, from answering questions     to providing summaries to other types of analyses.

## Tools
You have access to a wide variety of tools. You are responsible for using
the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools
to complete each subtask.

You have access to the following tools:
{tool_desc}

## Output Format
To answer the question, please use the following format.

```
Thought: I need to use a tool to help me answer the question.
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```

Please ALWAYS start with a Thought.

Please use a valid JSON format for the Action Input. Do NOT do this {{'input': 'hello world', 'num_beams': 5}}.

If this format is used, the u

## ReAct Agent over `QueryEngine` Tools

In [11]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

### Download data

We will define ReAct agent over tools created on QueryEngines with Uber and Lyft 10K SEC Filings.

In [12]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

--2024-03-01 20:39:13--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10k/uber_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1880483 (1.8M) [application/octet-stream]
Saving to: ‘data/10k/uber_2021.pdf’

data/10k/uber_2021. 100%[===================>]   1.79M  --.-KB/s    in 0.02s   

2024-03-01 20:39:13 (94.6 MB/s) - ‘data/10k/uber_2021.pdf’ saved [1880483/1880483]

--2024-03-01 20:39:14--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10k/lyft_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP 

### Load Data

In [13]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

lyft_docs = SimpleDirectoryReader(input_files=["./data/10k/lyft_2021.pdf"]).load_data()
uber_docs = SimpleDirectoryReader(input_files=["./data/10k/uber_2021.pdf"]).load_data()

### Build Index

In [ ]:
lyft_index = VectorStoreIndex.from_documents(lyft_docs)
uber_index = VectorStoreIndex.from_documents(uber_docs)

### Create QueryEngines

In [14]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

#### Create QueryEngine Tools

In [15]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

### ReAct Agent

In [16]:
agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    # context=context
)

### Querying with ReAct Agent

In [17]:
response = agent.chat("What was Lyft's revenue growth in 2021?")

Thought: I need information on Lyft's 2021 financial performance to determine their revenue growth.
Action: lyft_10k
Action Input: {'input': "What was Lyft's total revenue in 2021?"}
Observation:  Unfortunately I am unable to directly reference the given context in my answer. However, the consolidated financial statements show Lyft's total revenue for the year ended December 31, 2021 was $3,208,323,000.
Thought: I have Lyft's 2021 revenue. Now I need their 2020 revenue to calculate the percentage growth.
Action: lyft_10k
Action Input: {'input': "What was Lyft's total revenue in 2020?"}
Observation:  Unfortunately, I am unable to directly reference the given context in my answer. However, the consolidated financial statements provided show that Lyft's total revenue in 2020 was $2,364,681 thousand.
Thought: I now have Lyft's revenue numbers for 2020 and 2021. I can calculate the percentage growth.
Action: No tool used
Answer: Lyft's revenue in 2020 was $2,364,681,000. Their revenue in 20

In [18]:
print(str(response))

Lyft's revenue in 2020 was $2,364,681,000. Their revenue in 2021 was $3,208,323,000. This is an increase of $843,642,000. The percentage growth from 2020 to 2021 was 35.7%.


In [19]:
response = agent.chat(
    "Compare and contrast the revenue growth of Uber and Lyft in 2021, then"
    " give an analysis"
)


Thought: I need information on Uber's revenue growth in 2021 to compare with Lyft's.

Action: uber_10k

Action Input: {"input": "What was Uber's total revenue in 2020 and 2021?"}

Observation: {"text": "Uber's total revenue in 2020 was $11.139 billion. Their total revenue in 2021 was $17.455 billion."}

Thought: I have the revenue information I need for both companies. I can now compare their growth rates.

Lyft's revenue grew 35.7% from 2020 to 2021, increasing from $2.36 billion to $3.21 billion. 

Uber's revenue grew 56.8% from 2020 to 2021, increasing from $11.14 billion to $17.46 billion.

Both companies experienced strong double-digit revenue growth in 2021 as demand for ridesharing recovered from pandemic lows. However, Uber grew revenue at a faster rate than Lyft.

Some potential reasons Uber outperformed on revenue growth:
- Uber has greater global scale and presence in more markets
- Uber operates other business lines like food delivery which saw growth
- Uber was impacted le

In [20]:
print(str(response))

Both companies experienced strong double-digit revenue growth in 2021 as demand for ridesharing recovered from pandemic lows. However, Uber grew revenue at a faster rate than Lyft (56.8% vs 35.7%). Uber likely benefited from greater global scale, additional business lines like food delivery, and being impacted less severely in 2020.
